In [1]:
#pip install openpyxl


In [2]:
import pandas as pd

In [3]:
xls = pd.ExcelFile('Supply chain logistics problem.xlsx')

In [4]:
sheets = xls.sheet_names
dfs = [xls.parse(sheet) for sheet in sheets]

In [5]:
dfs

[          Order ID Order Date Origin Port Carrier  TPT Service Level  \
 0     1.447296e+09 2013-05-26      PORT09   V44_3    1           CRF   
 1     1.447158e+09 2013-05-26      PORT09   V44_3    1           CRF   
 2     1.447139e+09 2013-05-26      PORT09   V44_3    1           CRF   
 3     1.447364e+09 2013-05-26      PORT09   V44_3    1           CRF   
 4     1.447364e+09 2013-05-26      PORT09   V44_3    1           CRF   
 ...            ...        ...         ...     ...  ...           ...   
 9210  1.447305e+09 2013-05-26      PORT04  V444_1    1           DTD   
 9211  1.447319e+09 2013-05-26      PORT04  V444_1    1           DTD   
 9212  1.447322e+09 2013-05-26      PORT04  V444_1    1           DTD   
 9213  1.447145e+09 2013-05-26      PORT04  V444_1    1           DTD   
 9214  1.447328e+09 2013-05-26      PORT04  V444_1    1           DTD   
 
       Ship ahead day count  Ship Late Day count           Customer  \
 0                        3                    0   

In [6]:
dfs[0].fillna(0, inplace=True)
dfs[1].fillna(0, inplace=True)
dfs[2].fillna(0, inplace=True)
dfs[3].fillna(0, inplace=True)
dfs[4].fillna(0, inplace=True)
dfs[5].fillna(0, inplace=True)
dfs[6].fillna(0, inplace=True)

In [7]:
sheet_names = ['OrderList', 'FreightRates', 'WhCosts', 'WhCapacities', 'ProductsPerPlant', 'VmiCustomers', 'PlantPorts']
for sheet in sheet_names:
    df = xls.parse(sheet)
    print(f"Columns in {sheet}: {df.columns.tolist()}\n")


Columns in OrderList: ['Order ID', 'Order Date', 'Origin Port', 'Carrier', 'TPT', 'Service Level', 'Ship ahead day count', 'Ship Late Day count', 'Customer', 'Product ID', 'Plant Code', 'Destination Port', 'Unit quantity', 'Weight']

Columns in FreightRates: ['Carrier', 'orig_port_cd', 'dest_port_cd', 'minm_wgh_qty', 'max_wgh_qty', 'svc_cd', 'minimum cost', 'rate', 'mode_dsc', 'tpt_day_cnt', 'Carrier type']

Columns in WhCosts: ['WH', 'Cost/unit']

Columns in WhCapacities: ['Plant ID', 'Daily Capacity ']

Columns in ProductsPerPlant: ['Plant Code', 'Product ID']

Columns in VmiCustomers: ['Plant Code', 'Customers']

Columns in PlantPorts: ['Plant Code', 'Port']



In [8]:
dfs[0] = pd.get_dummies(dfs[0], columns=['Order ID', 'Order Date', 'Origin Port', 'Carrier', 'TPT', 'Service Level', 'Ship ahead day count', 'Ship Late Day count', 'Customer', 'Product ID', 'Plant Code', 'Destination Port', 'Unit quantity', 'Weight'])
dfs[1] = pd.get_dummies(dfs[1], columns=['Carrier', 'orig_port_cd', 'dest_port_cd', 'minm_wgh_qty', 'max_wgh_qty', 'svc_cd', 'minimum cost', 'rate', 'mode_dsc', 'tpt_day_cnt', 'Carrier type'])
dfs[2] = pd.get_dummies(dfs[2], columns=['WH', 'Cost/unit'])
dfs[3] = pd.get_dummies(dfs[3], columns=['Plant ID', 'Daily Capacity '])
dfs[4] = pd.get_dummies(dfs[4], columns=['Plant Code', 'Product ID'])
dfs[5] = pd.get_dummies(dfs[5], columns=['Plant Code', 'Customers'])
dfs[6] = pd.get_dummies(dfs[6], columns=['Plant Code', 'Port'])

In [9]:
import numpy as np

In [11]:
sheet_names = ['OrderList', 'FreightRates', 'WhCosts', 'WhCapacities', 'ProductsPerPlant', 'VmiCustomers', 'PlantPorts']
columns = {
    'OrderList': ['Ship ahead day count', 'Ship Late Day count', 'Unit quantity', 'Weight'],
    'FreightRates': ['minm_wgh_qty', 'max_wgh_qty', 'minimum cost', 'rate', 'tpt_day_cnt'],
    'WhCosts': ['Cost/unit'],
    'WhCapacities': ['Daily Capacity '],
    'ProductsPerPlant': ['Product ID'],
}
for sheet in sheet_names:
    df = xls.parse(sheet)
    
    for column in columns.get(sheet, []):
        # Calculate IQR
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        
        lower_limit = Q1 - 1.5 * IQR
        upper_limit = Q3 + 1.5 * IQR
        
        df = df[(df[column] >= lower_limit) & (df[column] <= upper_limit)]
        
    print(f"Cleaned DataFrame for {sheet}:\n{df}\n")
        
    
        


Cleaned DataFrame for OrderList:
          Order ID Order Date Origin Port Carrier  TPT Service Level  \
0     1.447296e+09 2013-05-26      PORT09   V44_3    1           CRF   
3     1.447364e+09 2013-05-26      PORT09   V44_3    1           CRF   
7     1.447158e+09 2013-05-26      PORT09   V44_3    1           CRF   
11    1.447171e+09 2013-05-26      PORT09   V44_3    1           CRF   
13    1.447257e+09 2013-05-26      PORT09   V44_3    1           CRF   
...            ...        ...         ...     ...  ...           ...   
9210  1.447305e+09 2013-05-26      PORT04  V444_1    1           DTD   
9211  1.447319e+09 2013-05-26      PORT04  V444_1    1           DTD   
9212  1.447322e+09 2013-05-26      PORT04  V444_1    1           DTD   
9213  1.447145e+09 2013-05-26      PORT04  V444_1    1           DTD   
9214  1.447328e+09 2013-05-26      PORT04  V444_1    1           DTD   

      Ship ahead day count  Ship Late Day count           Customer  \
0                        3      

In [16]:
for sheet in sheet_names:
    df = xls.parse(sheet)
    df.to_csv(f'preprocessed_{sheet}.csv', index=False)

In [14]:
#pip install xlsxwriter



     -------------------------------------- 159.9/159.9 kB 3.2 MB/s eta 0:00:00
